#Maestria en "Analítica de Datos"
---

Nombre: Oswaldo Salgado Gómez

Código: 79133277

Fecha: 2025 agosto 28

Descripción: Este libro trabaja ejercicios de procesamiento en paralelo

#1.Cargar librerias

In [13]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
url_folder_csv="/content/drive/MyDrive/3. Semestre 3/Big Data/Talleres/T_2"

#2. Carga/Obtención de la información

In [ ]:
listado_archivos=[f for f in os.listdir(url_folder_csv) if f.endswith (".csv")]
print(listado_archivos)


['tutelas_con_providencias_2021.csv', 'tutelas_con_providencias_2022.csv', 'tutelas_con_providencias_2025.csv', 'tutelas_con_providencias_2023.csv', 'tutelas_con_providencias_2024.csv']


In [ ]:
columnas_nombre=["clasificacion",
                 "num_expediente",
                 "fecha_radica",
                 "demandante",
                 "demandado",
                 "primera_instancia",
                 "segunda_instancia",
                 "num_23Digitos",
                 "providencia",
                 "fecha_setencia"]
df_tutelas=pd.DataFrame(columns=columnas_nombre)
display(df_tutelas)

,clasificacion,num_expediente,fecha_radica,demandante,demandado,primera_instancia,segunda_instancia,num_23Digitos,providencia,fecha_setencia


##2.a Procesamiento secuencial (uno tras otro)

In [ ]:
df_list=[] #Lista de df resultante de los datos de cada uno de los csv
for archivo_nombre in listado_archivos:
    ruta_archivo=os.path.join(url_folder_csv, archivo_nombre) #ruta completa del archivo
    print(ruta_archivo)
    df_temporal=pd.read_csv(ruta_archivo,sep=";",header=None, encoding="latin-1", on_bad_lines="skip")
    df_temporal.columns=columnas_nombre
    if (len(df_temporal.columns)==len(columnas_nombre)):
        df_list.append(df_temporal)
        print(f" archivo{archivo_nombre} carga exitosamente")
    else:
        print(f"archivo no trabajado {archivo_nombre} no tiene 10 columnas")

# una vexz termine de cargar todos csv combinelos en el dataframe df_tutelas
if df_list:
    df_tutelas=pd.concat(df_list, ignore_index=True)
    #remplazar las palabras "NULL" por None
    df_tutelas.replace("NULL", None,inplace=True)
    display(df_tutelas.head())


/content/drive/MyDrive/3. Semestre 3/Big Data/Talleres/T_2/tutelas_con_providencias_2021.csv
 archivotutelas_con_providencias_2021.csv carga exitosamente
/content/drive/MyDrive/3. Semestre 3/Big Data/Talleres/T_2/tutelas_con_providencias_2022.csv
 archivotutelas_con_providencias_2022.csv carga exitosamente
/content/drive/MyDrive/3. Semestre 3/Big Data/Talleres/T_2/tutelas_con_providencias_2025.csv


/tmp/ipython-input-1435501799.py:5: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_temporal=pd.read_csv(ruta_archivo,sep=";",header=None, encoding="latin-1", on_bad_lines="skip")


 archivotutelas_con_providencias_2025.csv carga exitosamente
/content/drive/MyDrive/3. Semestre 3/Big Data/Talleres/T_2/tutelas_con_providencias_2023.csv
 archivotutelas_con_providencias_2023.csv carga exitosamente
/content/drive/MyDrive/3. Semestre 3/Big Data/Talleres/T_2/tutelas_con_providencias_2024.csv
 archivotutelas_con_providencias_2024.csv carga exitosamente


,clasificacion,num_expediente,fecha_radica,demandante,demandado,primera_instancia,segunda_instancia,num_23Digitos,providencia,fecha_setencia
0,ï»¿Acciones de Tutela,T8531729,2021-12-16 00:00:00,VAZQUEZ CUADROS BAYRON EMILIO Y OTROS EN REPRE...,SOCIEDAD DE ACTIVOS ESPECIALES S.A.E. S.A.S.,"BOGOTA,CUNDINAMARCA, TRIBUNAL SUPERIOR DE BOGO...",--,--,--,NaN
1,Acciones de Tutela,T8531726,2021-12-16 00:00:00,ESCORCIA JORGE ARMANDO,ALCALDIA MUNICIPAL DE SANTA LUCIA,"SANTA LUCIA,ATLANTICO, JUZGADO 1 PROMISCUO MUN...","SABANALARGA,ATLANTICO, JUZGADO 2 PROMISCUO DEL...",--,--,NaN
2,Acciones de Tutela,T8531728,2021-12-16 00:00:00,ARCINIEGAS MEDRANO CLEMENCIA,COLPENCIONES Y OTRO,"BARRANQUILLA,ATLANTICO, JUZGADO 11 LABORAL DEL...","BARRANQUILLA,ATLANTICO, TRIBUNAL SUPERIOR SALA...",--,T-274/22,2022-07-28 00:00:00.000
3,Acciones de Tutela,T8531723,2021-12-16 00:00:00,SERRANO ROJAS CAMILO ANDRES,SECRETARIA DE MOVILIDAD DE CAQUEZA,"CAQUEZA,CUNDINAMARCA, JUZGADO 1 PROMISCUO MUNI...",--,--,--,NaN
4,Acciones de Tutela,T8531727,2021-12-16 00:00:00,GOMEZ QUINTERO EDILSA MARIA,CAJACOPI EPS Y OTRO,"CURUMANI,CESAR, JUZGADO PROMISCUO MUNICIPAL","CHIRIGUANA,CESAR, JUZGADO PENAL DEL CIRCUITO",--,--,NaN


##2.b Procesamiento en paralelo (todos al tiempo)

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed

In [ ]:
def cargar_data_desde_archivo_csv(ruta_archivo, archivo_nombre):
    try:
        df_temporal=pd.read_csv(ruta_archivo,sep=";",header=None, encoding="latin-1", on_bad_lines="skip")
        #Agregar
        df_temporal.columns=columnas_nombre
        if (len(df_temporal.columns)==len(columnas_nombre)):
            print(f" archivo{archivo_nombre} carga exitosamente")
            return df_temporal
        else:
            print(f"archivo no trabajado {archivo_nombre} no tiene 10 columnas")
            return None

    except Exception as e:
        print(f"Error al procesar el archivo {ruta_archivo}: {str(e)}")
        return None

In [ ]:
sd_list=[] #lista de df resultante de los datos de cada uno de los csv
with ThreadPoolExecutor() as executor:
    future_to_file={executor.submit(cargar_data_desde_archivo_csv, os.path.join(url_folder_csv, archivo_nombre), archivo_nombre): archivo_nombre for archivo_nombre in listado_archivos}
    for future in as_completed(future_to_file):
        df=future.result()
        if df is not None:
            sd_list.append(df)

# Concatenate all dataframes in the list
if sd_list:
    df_tutelas_parallel = pd.concat(sd_list, ignore_index=True)
    # Replace "NULL" with None
    df_tutelas_parallel.replace("NULL", None, inplace=True)
    display(df_tutelas_parallel.head())

 archivotutelas_con_providencias_2021.csv carga exitosamente
 archivotutelas_con_providencias_2022.csv carga exitosamente


/tmp/ipython-input-1706949637.py:3: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_temporal=pd.read_csv(ruta_archivo,sep=";",header=None, encoding="latin-1", on_bad_lines="skip")


 archivotutelas_con_providencias_2025.csv carga exitosamente
 archivotutelas_con_providencias_2023.csv carga exitosamente
 archivotutelas_con_providencias_2024.csv carga exitosamente


,clasificacion,num_expediente,fecha_radica,demandante,demandado,primera_instancia,segunda_instancia,num_23Digitos,providencia,fecha_setencia
0,ï»¿Acciones de Tutela,T8531729,2021-12-16 00:00:00,VAZQUEZ CUADROS BAYRON EMILIO Y OTROS EN REPRE...,SOCIEDAD DE ACTIVOS ESPECIALES S.A.E. S.A.S.,"BOGOTA,CUNDINAMARCA, TRIBUNAL SUPERIOR DE BOGO...",--,--,--,NaN
1,Acciones de Tutela,T8531726,2021-12-16 00:00:00,ESCORCIA JORGE ARMANDO,ALCALDIA MUNICIPAL DE SANTA LUCIA,"SANTA LUCIA,ATLANTICO, JUZGADO 1 PROMISCUO MUN...","SABANALARGA,ATLANTICO, JUZGADO 2 PROMISCUO DEL...",--,--,NaN
2,Acciones de Tutela,T8531728,2021-12-16 00:00:00,ARCINIEGAS MEDRANO CLEMENCIA,COLPENCIONES Y OTRO,"BARRANQUILLA,ATLANTICO, JUZGADO 11 LABORAL DEL...","BARRANQUILLA,ATLANTICO, TRIBUNAL SUPERIOR SALA...",--,T-274/22,2022-07-28 00:00:00.000
3,Acciones de Tutela,T8531723,2021-12-16 00:00:00,SERRANO ROJAS CAMILO ANDRES,SECRETARIA DE MOVILIDAD DE CAQUEZA,"CAQUEZA,CUNDINAMARCA, JUZGADO 1 PROMISCUO MUNI...",--,--,--,NaN
4,Acciones de Tutela,T8531727,2021-12-16 00:00:00,GOMEZ QUINTERO EDILSA MARIA,CAJACOPI EPS Y OTRO,"CURUMANI,CESAR, JUZGADO PROMISCUO MUNICIPAL","CHIRIGUANA,CESAR, JUZGADO PENAL DEL CIRCUITO",--,--,NaN


In [ ]:
# Número de Tutelas por Juzgado
tutelas_por_juzgado = df_tutelas_parallel.groupby('primera_instancia').size().reset_index(name='Número de Casos')

tutelas_por_juzgado = tutelas_por_juzgado.rename(columns={'primera_instancia': 'Juzgado'})

display(tutelas_por_juzgado)

,Juzgado,Número de Casos
0,", ,",5
1,", JUZGADO 006 CIVIL MUNICIPAL DE CARTAGENA",1
2,", JUZGADO 1 PENAL MUNICIPAL PARA ADOLESCENTES ...",1
3,", JUZGADO 2 CIVIL MUNICIPAL",1
4,", JUZGADO 3 PENAL MUNICIPAL CON FUNCIONES MIXTAS",1
...,...,...
115051,"pereira,risaralda, JUZGADO 007 PENAL DEL CIRCU...",1
115052,prueba,1
115053,"ÂºTULUA,VALLE DEL CAUCA, JUZGADO 2 PENAL MUNIC...",1
115054,"ÃBREGO,NORTE DE SANTANDER, JUZGADO PROMISCUO ...",11


In [ ]:
tutelas_2021_al_2025=df_tutelas_parallel

In [14]:
import os

# Define la ruta donde se guardará el archivo CSV en tu Google Drive
ruta_salida = "/content/drive/MyDrive/3. Semestre 3/Big Data/Talleres/T_2/Consolidado_Tutelas/tutelas_2021_al_2025.csv"

# Extrae la ruta del directorio de la ruta de salida
directorio_salida = os.path.dirname(ruta_salida)

# Crea el directorio si no existe
if not os.path.exists(directorio_salida):
    os.makedirs(directorio_salida)

# Guarda el DataFrame en un archivo CSV
tutelas_2021_al_2025.to_csv(ruta_salida, index=False)

print(f"DataFrame guardado en {ruta_salida}")

DataFrame guardado en /content/drive/MyDrive/3. Semestre 3/Big Data/Talleres/T_2/Consolidado_Tutelas/tutelas_2021_al_2025.csv
